# Modeling the emergence of adj-noun ordering preferences

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Define objects and utterances

In [2]:
adjectives = ['red', 'blue']
nouns = ['pin', 'dress']

Let's define a function to create all the possible objects in the world.

In [4]:
def get_all_objects(adjectives, nouns):
    objects = []
    for adj in adjectives:
        for noun in nouns:
            obj = {}
            obj["color"] = adj
            obj["shape"] = noun
            obj["string"] = adj + " " + noun
            objects.append(obj)
            
    return objects

Now we can see all the objects:

In [5]:
objects = get_all_objects(adjectives, nouns)
objects

[{'color': 'red', 'shape': 'pin', 'string': 'red pin'},
 {'color': 'red', 'shape': 'dress', 'string': 'red dress'},
 {'color': 'blue', 'shape': 'pin', 'string': 'blue pin'},
 {'color': 'blue', 'shape': 'dress', 'string': 'blue dress'}]

Next, we can define all the possible utterances. Similar to above, we'll write a function to compressly express all the combos.

In [6]:
def get_all_utterances(adjectives, nouns):
    utterances = []
    utterances.extend(adjectives)
    utterances.extend(nouns)
    for adj in adjectives:
        for noun in nouns:
            adj_first = adj + " " + noun
            noun_first = noun + " " + adj
            utterances.append(adj_first)
            utterances.append(noun_first)
            
    return utterances

In [7]:
utterances = get_all_utterances(adjectives, nouns)
utterances

['red',
 'blue',
 'pin',
 'dress',
 'red pin',
 'pin red',
 'red dress',
 'dress red',
 'blue pin',
 'pin blue',
 'blue dress',
 'dress blue']

Now, we're going to define our meaning function, which is slightly different from how we did it in the simple case. This meaning function maps an utterance to every possible object for which that utterance applies, taking into account multi-word utterances (so ``blue square`` could only refer to an object that is both blue and square, not, for example, any blue thing).

Additionally, the meaning function will (eventually) also capture asymmetries in referential noise between adjectives and nouns, but we'll hold off on that for now.

In [8]:
def meaning(obj, utt):
    utt_as_tokens = utt.split()
    all_contained = set(utt_as_tokens).issubset(obj.values())
            
    return int(all_contained)

In [9]:
meaning({'color': 'red', 'shape': 'square', 'string': 'red square'}, 'square')

1

## Continuous global speaker

#### Literal listener

In [10]:
def normalize_rows(matrix):
    """
    Helper function that normalize probabilities across rows to sum to 1
    """
    totals = np.sum(matrix, axis=1)
    return matrix / totals[:, np.newaxis]

def literal_listener(utt, meaning_fn=meaning):
    # generate the matrix of utterances x world states
    all_counts = np.zeros(shape=(len(utterances), len(objects)))
    for i in range(len(utterances)):
        for j in range(len(objects)):
            curr_utt = utterances[i]
            curr_obj = objects[j]

            all_counts[i, j] = meaning_fn(curr_obj, curr_utt)
            # if I wanted to incorporate a prior I would do it here
                
    data = normalize_rows(all_counts)
    df_cols = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=df_cols, index=utterances)
    return df, df.loc[utt]

In [11]:
l0_df, l0_probs = literal_listener('red')

In [12]:
l0_probs

red pin       0.5
red dress     0.5
blue pin      0.0
blue dress    0.0
Name: red, dtype: float64

#### Continuous meaning function

In [13]:
def token_applies_to_obj(obj, token):
    return token in obj.values()

def continuous_meaning(obj, utt):
    v_adj = 0.95
    v_noun = 0.99
    total = 1
    for token in utt.split():
        if token_applies_to_obj(obj, token):
            if token in adjectives: total *= v_adj
            if token in nouns: total *= v_noun
        else:
            if token in adjectives: total *= (1 - v_adj)
            if token in nouns: total *= (1 - v_noun)
                
    return total

In [14]:
continuous_meaning({'color': 'red', 'shape': 'pin', 'string': 'red pin'}, 'red pin')

0.9405

#### Cost function

In [15]:
def cost(utt):
    return 0.1 * len(utt.split())

#### Pragmatic speaker

In [16]:
def pragmatic_speaker(obj, alpha=1):
    all_vals = []
    for curr_utt in utterances:
        _, probs = literal_listener(curr_utt, meaning_fn=continuous_meaning)
        utility = np.array(probs)
        val = np.exp(alpha * (np.log(utility) - cost(curr_utt)))
        all_vals.append(val)
        
    data = normalize_rows(np.array(all_vals).T)
    df_idx = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=utterances, index=df_idx)
    
    return df, df.loc[obj["string"]]

In [17]:
s1_df, s1_probs = pragmatic_speaker({'color': 'red', 'shape': 'pin', 'string': 'red pin'}, alpha=3)
s1_df

,red,blue,pin,dress,red pin,pin red,red dress,dress red,blue pin,pin blue,blue dress,dress blue
red pin,0.073343,0.000011,8.300280e-02,8.554353e-08,4.217598e-01,4.217598e-01,4.346700e-07,4.346700e-07,6.148999e-05,6.148999e-05,6.337221e-11,6.337221e-11
red dress,0.073343,0.000011,8.554353e-08,8.300280e-02,4.346700e-07,4.346700e-07,4.217598e-01,4.217598e-01,6.337221e-11,6.337221e-11,6.148999e-05,6.148999e-05
blue pin,0.000011,0.073343,8.300280e-02,8.554353e-08,6.148999e-05,6.148999e-05,6.337221e-11,6.337221e-11,4.217598e-01,4.217598e-01,4.346700e-07,4.346700e-07
blue dress,0.000011,0.073343,8.554353e-08,8.300280e-02,6.337221e-11,6.337221e-11,6.148999e-05,6.148999e-05,4.346700e-07,4.346700e-07,4.217598e-01,4.217598e-01


In [18]:
s1_probs

red           7.334289e-02
blue          1.069294e-05
pin           8.300280e-02
dress         8.554353e-08
red pin       4.217598e-01
pin red       4.217598e-01
red dress     4.346700e-07
dress red     4.346700e-07
blue pin      6.148999e-05
pin blue      6.148999e-05
blue dress    6.337221e-11
dress blue    6.337221e-11
Name: red pin, dtype: float64

In this setup, the utterances 'blue pin' and 'pin blue' have the same probability, which is not what we want. Thus we move to the incremental setting.

## Incremental continuous speaker

This gets us to the point of Brandon's squib.

#### Incremental continuous meaning function

According to Cohn-Gordon (2019):
"For any partial sequence $c$ and set of referents $W$, $[[c]](w) \in [0, 1]$ is the number of full-utterance extensions of $c$ true in $w$ divided by the number of possible extensions of $c$ into full utterances that are true of any world in $W$."

In [19]:
def inc_cont_meaning(obj, utt):
    num_true_extensions = []
    num_possible_extensions = []
    
    utility = 0
    for curr_utt in utterances:
        
        utt_starts_with = curr_utt.startswith(utt)
        
        if utt_starts_with:
            num_possible_extensions.append(curr_utt)
            
            curr_utt_as_tokens = curr_utt.split()
            utility += continuous_meaning(obj, utt)
            
    return utility/len(num_possible_extensions)

In [20]:
inc_cont_meaning({'color': 'red', 'shape': 'pin', 'string': 'red pin'}, 'red')

0.9499999999999998

#### Word-level literal listener

This is pretty much exactly the same as the standard literal listener, except it should only accept a single word as an argument (and maybe a log transform? TBD).

In [21]:
def word_level_literal_listener(word, meaning_fn):
    # generate the matrix of utterances x world states
    all_counts = np.zeros(shape=(len(utterances), len(objects)))
    for i in range(len(utterances)):
        for j in range(len(objects)):
            curr_utt = utterances[i]
            curr_obj = objects[j]

            # take the log here(?)
            all_counts[i, j] = np.log(meaning_fn(curr_obj, curr_utt))
    
    data = normalize_rows(all_counts)
    df_cols = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=df_cols, index=utterances)
    return df, df.loc[word]

In [22]:
df_word_level_l0, _ = word_level_literal_listener('red', inc_cont_meaning)
df_word_level_l0

,red pin,red dress,blue pin,blue dress
red,0.008417,0.008417,0.491583,0.491583
blue,0.491583,0.491583,0.008417,0.008417
pin,0.001089,0.498911,0.001089,0.498911
dress,0.498911,0.001089,0.498911,0.001089
red pin,0.004003,0.303858,0.196142,0.495997
pin red,0.004003,0.303858,0.196142,0.495997
red dress,0.303858,0.004003,0.495997,0.196142
dress red,0.303858,0.004003,0.495997,0.196142
blue pin,0.196142,0.495997,0.004003,0.303858
pin blue,0.196142,0.495997,0.004003,0.303858


#### Word-level pragmatic speaker

First, we define the helper function `get_possible_completions`:

In [23]:
def get_possible_completions(utt):
    """
    Given a partial utterance, return all the possible completions
    """
    max_utt_len = len(utt.split()) + 1
    
    possible_completions = []
    for curr_utt in utterances:
        if curr_utt.startswith(utt) and len(curr_utt.split()) <= max_utt_len:
            possible_completions.append(curr_utt)
            
    return possible_completions

In [24]:
get_possible_completions('red')

['red', 'red pin', 'red dress']

Now we can define the word-level pragmatic speaker. The key difference here between this and standard RSA is that you are only iterating over the utterances that are possible completions of the context, and _not_ all possible utterances. Thus, we need to pass in `context` as an argument.

In [25]:
def word_level_pragmatic_speaker(obj, context, alpha=1):
    all_vals = []
    possible_utterances = get_possible_completions(context)
    for curr_utt in possible_utterances:
        _, probs = literal_listener(curr_utt, meaning_fn=inc_cont_meaning) # specify meaning_fn here
        utility = np.array(probs)
        val = np.exp(alpha * np.log(utility) - cost(curr_utt))
        all_vals.append(val)

    data = normalize_rows(np.array(all_vals).T)
    df_idx = [obj["string"] for obj in objects]
    df = pd.DataFrame(data, columns=possible_utterances, index=df_idx)
    
    return df, df.loc[obj["string"]]

In [26]:
df_word_level_s1, _ = word_level_pragmatic_speaker({'color': 'red', 'shape': 'pin', 'string': 'red pin'}, context = 'red')
df_word_level_s1

,red,red pin,red dress
red pin,0.355913,0.637646,0.006441
red dress,0.355913,0.006441,0.637646
blue pin,0.355913,0.637646,0.006441
blue dress,0.355913,0.006441,0.637646


In [27]:
def incremental_pragmatic_speaker(obj, utt):
    all_vals = []
    utt_len = len(utt.split())
    context = ''
    val = 1
    for i in range(utt_len):

        _, probs = word_level_pragmatic_speaker(obj, context, alpha=5)
        partial_utt = " ".join(utt.split()[:i+1])
        index_of_partial_utt = list(probs.index).index(partial_utt)

        val *= list(probs)[index_of_partial_utt]
        
        context = partial_utt

    return val

In [28]:
incremental_pragmatic_speaker({'color': 'red', 'shape': 'pin', 'string': 'red pin'}, 'red pin')

0.43290650693812754

In [29]:
incremental_pragmatic_speaker({'color': 'red', 'shape': 'pin', 'string': 'red pin'}, 'pin red')

0.5278132858034454